In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
# Assuming input is x, and x is already padded
c = channel
h = height
w = width
ker = kernel
s = stride
pad = padding

output_height = (h - ker + 2*pad) // s + 1
output_width = (w - ker + 2*pad) // s + 1

k = input_channels * kernel_height * kernel_width  # Flattened patch size
p = output_height * output_width  # Total number of patches
unfolded = zeros(batch_size, k, p)  # Output tensor

for b in range(batch_size):  # Iterate over batches
    for oh in range(output_height):  # Iterate over output height
        for ow in range(output_width):  # Iterate over output width
            patch_flattened = []  # Temporary list to hold patch values

            h_start = oh * stride
            w_start = ow * stride
            for ic in range(input_channels):  # Iterate over input channels
                for kh in range(kernel_height):  # Iterate over kernel height
                    for kw in range(kernel_width):  # Iterate over kernel width
                        patch_flattened.append(
                            padded_x[b, ic, h_start + kh, w_start + kw]
                        )
            # Store the flattened patch in the output tensor
            patch_index = oh * output_width + ow
            unfolded[b, :, patch_index] = patch_flattened

# so the output tensor is of shape (batch_size, k, p)


NameError: name 'channel' is not defined

In [130]:
import torch
import torch.nn.functional as F
from torch import nn
class Conv2D(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size=(3,3),stride=1, padding=1):
        super(Conv2D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(out_channels, *kernel_size))
        self.bias = nn.Parameter(torch.randn(out_channels))
    def forward(self, input_batch):
        b, c, h, w = input_batch.size()
        unfolded = F.unfold(input_batch, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
        print(f'unfolded size is: {unfolded.size()}')
        
        # Reshape unfolded input to (b * p, k)
        unfolded = unfolded.permute(0, 2, 1).reshape(b * unfolded.size(2), -1)
        print(f'unfolded size is: {unfolded.size()}')
        
        # Reshape weight to (out_channels, in_channels * kernel_size[0] * kernel_size[1])
        weight = self.weight.view(self.out_channels, -1)
        print(f'weight size is: {weight.size()}')
        
        # Perform matrix multiplication
        output = torch.mm(unfolded, weight.t()).view(b, -1, unfolded.size(1))
        
        # Add bias and reshape output to (b, out_channels, output_h, output_w)
        output = output + self.bias.view(1, -1, 1)
        output_h = (h + 2 * self.padding - self.kernel_size[0]) // self.stride + 1
        output_w = (w + 2 * self.padding - self.kernel_size[1]) // self.stride + 1
        output = output.view(b, self.out_channels, output_h, output_w)
        
        return output        # apply the convolution operation
conv = Conv2D(3, 3)
input_batch = torch.randn(16, 3, 32, 32)

output_batch = conv(input_batch)
output_batch
print(input_batch.size())
conv.weight.size()

unfolded size is: torch.Size([16, 27, 1024])
unfolded size is: torch.Size([16384, 27])
weight size is: torch.Size([3, 9])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16384x27 and 9x3)

In [82]:
print(input_batch.size())
print(output_batch.size())

torch.Size([16, 3, 32, 32])


AttributeError: 'NoneType' object has no attribute 'size'

In [ ]:
print(conv.weight)

Parameter containing:
tensor([[[-0.0798,  0.9788,  1.1968],
         [-0.6824,  1.3893, -1.3727],
         [-0.6268, -2.0099, -0.3287]],

        [[ 1.4774, -2.4945,  0.3863],
         [-0.5913, -0.4336, -0.4694],
         [-0.5908, -0.3391, -2.3395]],

        [[ 0.7544, -0.0415, -0.3136],
         [ 0.0282, -1.0349, -1.5563],
         [-0.9184, -0.8614,  0.2192]]], requires_grad=True)
